# Simultaneous global and target analysis of the timeresolved fluorescence of WL-PSI and FRL-PSI of CF9212

### Defining datasets and inspect data

In [ ]:
from __future__ import annotations

from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme
from pyglotaran_extras.inspect import show_a_matrixes
from pyglotaran_extras.plotting.style import PlotStyle
from pyglotaran_extras import (
    plot_overview,
    plot_data_overview,
    plot_fitted_traces,
    select_plot_wavelengths,
)


The code below defined the (groups of) datasets used in the analysis. Only for the three FRL datasets the plot_data_overview is shown to avoid repetition, it is left as an exercise to the reader to inspect the WL data.

In [ ]:
# order FRLtr1, WLtr1, FRLtr2, WLtr2, FRLtr4
CF9212DATA_PATH1 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2a.ascii"
CF9212DATA_PATH2 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2b.ascii"
CF9212DATA_PATH3 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2c.ascii"
CF9212DATA_PATH4 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2d.ascii"
CF9212DATA_PATH5 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2e.ascii"
CF9212DATA_PATH6 = "data/PSI_9212FR_WLtr124av4nobridgetarget_nsuml2f.ascii"

# to plot uncomment the next lines with "Ctrl /"
plot_data_overview(
    CF9212DATA_PATH1,
    nr_of_data_svd_vectors=5,
    linlog=False,
    linthresh=10,
    irf_location=57,title="FRL-PSI Time Range 1",use_svd_number=True
);
# plot_data_overview(CF9212DATA_PATH2, nr_of_data_svd_vectors=5, linlog=True, linthresh=30,irf_location=57);
# etc


In [ ]:
plot_data_overview(
    CF9212DATA_PATH3,
    nr_of_data_svd_vectors=5,
    linlog=False,
    linthresh=10,
    irf_location=57,title="FRL-PSI Time Range 2",use_svd_number=True,
);


In [ ]:
plot_data_overview(
    CF9212DATA_PATH5,
    nr_of_data_svd_vectors=5,
    linlog=False,
    linthresh=100,
    irf_location=214,title="FRL-PSI Time Range 4",use_svd_number=True,
);


# Motivation of the model for the simultaneous global analysis of the FRL-PSI data

With all three time ranges the SVD points to a rank 5 matrix, which will require at least five temporally and spectrally linearly independent components. There is preknowledge that in FRL-PSI some free Chla and free Chlf are present, which have similar lifetimes of ~3 ns. Therefore we will start the global analysis with a sequential scheme with four compartments, plus two parallelly decaying free Chla and free Chlf compartments.
Analogous to WL-PSI we model the IRF with a double Gaussian shape and the dispersion with a 2nd order polynomial.
We will use independent free Chla and free Chlf fractions. To resolve the free Chla and free Chlf SAS we will use as a guidance the free Chla SAS estimated with WL-PSI, and a spectral constraint that the free Chlf SAS is zero below 700 nm.

## Optional: display model file and starting parameters

#### Model file

In [ ]:
# Uncomment the following 3 lines to display the target model file in the notebook
# from glotaran.utils.ipython import display_file
# target_model_path = "models/globalFRLstep1_PSI_CF9212_streak_global_dispFRL.yml"
# display_file(target_model_path, syntax="yaml")

# Alternatively (recommended), open the file in a text editor to see the model definition


#### Parameters file

In [ ]:
# Uncomment the last 3 lines and run the cell to print the starting values of the analysis
# These starting values have already been optimized, hence the name optimizedparameters
# target_parameters_path = "models/globalFRLstep1_PSI_CF9212_streak_global_dispFRL.csv"
# optimizedparameters = load_parameters(target_parameters_path)
# optimizedparameters


# Step 1 Create scheme for the simultaneous global analysis of the FRL-PSI data and optimize it

In [ ]:
target_scheme = Scheme(
    model="models/globalFRLstep1_PSI_CF9212_streak_global_dispFRL.yml",  # type: ignore
    parameters="models/globalFRLstep1_PSI_CF9212_streak_global_dispFRL.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212FRLtr1": CF9212DATA_PATH1,
        "CF9212FRLtr2": CF9212DATA_PATH3,
        "CF9212FRLtr4": CF9212DATA_PATH5,
        "freeChlaSAS": "data/20230912PSI_CF9212_streak_targetWL_disp_CF9212freeChla.ascii",
    },  # type: ignore
)
target_scheme.validate()


In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result1 = optimize(target_scheme, raise_exception=True)


For reference, the final Cost should be
-  (nfev) 7  (cost) 4.1479e+02

In [ ]:
target_result1


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_residual import plot_residual
from pyglotaran_extras.plotting.plot_svd import plot_lsv_residual
from pyglotaran_extras.plotting.plot_svd import plot_rsv_residual
from pyglotaran_extras.plotting.style import ColorCode as cc



def plot_svd_of_residual(result_dataset,result_dataset2,result_dataset3,linlog,linthresh,loc1,loc2,loc3,colors=["tab:grey","tab:orange",cc.cyan]):
    fig, axes = plt.subplots(1, 2, figsize=(10, 2))
    custom_cycler = cycler(color=[colors[0]])
    plot_lsv_residual(result_dataset, axes[0], indices=[0],linlog=linlog,linthresh=linthresh, cycler=custom_cycler,irf_location=loc1)
    custom_cycler = cycler(color=[colors[1]])
    plot_lsv_residual(result_dataset2, axes[0], indices=[0],linlog=linlog,linthresh=linthresh, cycler=custom_cycler,irf_location=loc2)
    custom_cycler = cycler(color=[colors[2]])
    plot_lsv_residual(result_dataset3, axes[0], indices=[0],linlog=linlog,linthresh=linthresh, cycler=custom_cycler,irf_location=loc3)
    axes[0].set_xlabel("Time (ps)")
    axes[0].get_legend().remove()
    axes[0].set_ylabel("")
    axes[0].set_title("residual 1st LSV")
    custom_cycler = cycler(color=[colors[0]])
    plot_rsv_residual(result_dataset, axes[1], indices=[0], cycler=custom_cycler)
    custom_cycler = cycler(color=[colors[1]])
    plot_rsv_residual(result_dataset2, axes[1], indices=[0], cycler=custom_cycler)
    custom_cycler = cycler(color=[colors[2]])
    plot_rsv_residual(result_dataset3, axes[1], indices=[0], cycler=custom_cycler)
    axes[1].set_xlabel("Wavelength (nm)")
    axes[1].set_title("residual 1st RSV")
    axes[1].get_legend().remove()
    axes[1].set_ylabel("")

    return fig, axes

fig, axes = plot_svd_of_residual(target_result1.data["CF9212FRLtr1"],target_result1.data["CF9212FRLtr2"],target_result1.data["CF9212FRLtr4"],linlog=True,linthresh=100,loc1=57,loc2=57,loc3=214)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


The residuals are structured, especially the RSV above 780 nm. but since there is no **common** trend, we can accept them.

In [ ]:
target_result1.root_mean_square_error


## Plot result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.plot_spectra import plot_das
from pyglotaran_extras.plotting.style import ColorCode
# ColorCode.green,"k",
myFRLcolors = [ "k","r","b","g",ColorCode.green,"tab:brown",   "tab:grey",ColorCode.cyan,  "w"]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.indigo, ColorCode.brown, "w","w","w","y",ColorCode.maroon]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
custom_cycler = cycler(color=myFRLcolors)
plot_concentrations(target_result1.data["CF9212FRLtr1"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(target_result1.data["CF9212FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_sas(target_result1.data["CF9212FRLtr4"], axes[1], cycler=custom_cycler)
plot_das(target_result1.data["CF9212FRLtr4"], axes[2], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[2].set_ylabel("")
axes[1].set_title("EAS/SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)


## Comparison of the estimated SAS (orange) and the guidance spectrum (blue)
The guidance spectrum of free Chla is the shape estimated in the WL target

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(5,5))
target_result1.data["freeChlaSAS"].data.plot(ax=axes)
target_result1.data["freeChlaSAS"].fitted_data.plot(ax=axes);
axes.set_xlabel("Wavelength (nm)")
axes.set_ylabel("")
axes.set_title("Bulk Chl a")


In [ ]:
show_a_matrixes(target_result1)


In [ ]:
target_result1.optimized_parameters


### Compute the FWHM of the main Gaussian of the IRF

In [ ]:
import numpy as np
const=2*np.sqrt(2*np.log(2))
[const*target_result1.optimized_parameters.get("CF9212irfFRLtr1.width1").value,
const*target_result1.optimized_parameters.get("CF9212irfFRLtr2.width1").value,
const*target_result1.optimized_parameters.get("CF9212irfFRLtr4.width1").value]


## Fit quality of the global analysis of the FRL data

In [ ]:
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        target_result1,
        [685,705,715,740,750,795],
        linlog=True,
        linthresh=10,
        axes_shape=(3, 2),
        figsize=(8,6),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.cyan,
                cc.blue,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "FRL TR1"
        elif i == 3:
            labels[i] = "FRL TR2"
        elif i == 5:
            labels[i] = "FRL TR4"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            # line.set_linewidth(0.5)  # Set the line width here
            line.set_linewidth(1)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()


# Step 2 Create target scheme and optimize it
maximum_number_function_evaluations=1 to compute only the precision in the minimum

In [ ]:
target_scheme = Scheme(
    model="models/targetFRL_WLstep2_PSI_CF9212_streak_target_dispRP2.yml",  # type: ignore
    parameters="models/targetFRL_WLstep2_PSI_CF9212_streak_target_dispRP2.csv",
    maximum_number_function_evaluations=7,
    clp_link_tolerance=0.1,
    data={
        "CF9212FRLtr1": CF9212DATA_PATH1,
        "CF9212WLtr1": CF9212DATA_PATH2,
        "CF9212FRLtr2": CF9212DATA_PATH3,
        "CF9212WLtr2": CF9212DATA_PATH4,
        "CF9212FRLtr4": CF9212DATA_PATH5,
        "CF9212WLtr4": CF9212DATA_PATH6,
        "BulkSAS": "data/20230912PSI_CF9212_streak_targetWL_disp_CF9212Bulk.ascii",
        "Red1SAS": "data/20230912PSI_CF9212_streak_targetWL_disp_CF9212Red1.ascii",
        "Red2SAS": "data/20230912PSI_CF9212_streak_targetWL_disp_CF9212Red2.ascii",
        "WLRCSAS": "data/20230912PSI_CF9212_streak_targetWL_disp_CF9212WLRC.ascii",
        "FRLRCSAS": "data/20230910PSI_CF9212_streak_target_disp_CF9212_FRLRC_fitted.ascii",
    },  # type: ignore
)
target_scheme.validate()


In [ ]:
# warning: this can take a minute or two, even on a fast machine
target_result = optimize(target_scheme, raise_exception=True)


For reference, the final Cost should be
-  7         5.7972e+02 with many parameters freed to compute the precision in the minimum

To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about **100 MB** of disk space per save.

In [ ]:
save_result(
    result=target_result,
    result_path="results/20230915target_disp/result.yaml",
    allow_overwrite=True,
)


In [ ]:
target_result2=target_result


In [ ]:
target_result2


In [ ]:
target_result1


## Print the estimates of the optimized parameters and their precision 
The t-values of the **free** parameters (Vary=True) indicate the precision

In [ ]:
target_result2.optimized_parameters


## Plot target WL+FRL result for interpretation


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode
# ColorCode.green,"k",
myFRLcolors = [ "g","tab:orange",  "r", "tab:grey",ColorCode.cyan, "b", ColorCode.green]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.indigo, ColorCode.brown,ColorCode.maroon, ColorCode.green,"y"]

fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
# plot_concentrations(target_result.data["CF9212WLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(target_result.data["CF9212WLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_concentrations(target_result.data["CF9212FRLtr4"], axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_sas(target_result.data["CF9212FRLtr4"], axes[1], cycler=custom_cycler2)
plot_sas(target_result.data["CF9212WLtr4"], axes[1], cycler=custom_cycler)
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SAS")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode
# ColorCode.green,"k",
# myFRLcolors = [ "g","tab:orange",  "r", "tab:grey",ColorCode.cyan,"b", "w", "w"]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.indigo, ColorCode.brown,ColorCode.maroon, "w","w","w","y"]
FRLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212FRLRC', 'CF9212Chlf1','CF9212Chlf2', 'CF9212FRLRP1',]
WLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212WLRC', 'CF9212WLRP1', ]
FRLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212FRLRC', 'CF9212Chlf1','CF9212Chlf2', 'CF9212FRLRP1', 'CF9212FRLRP2']
WLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212WLRC', 'CF9212WLRP1','CF9212WLRP2', ]
fig, axes = plt.subplots(1, 2, figsize=(15, 4))
custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
# plot_concentrations(target_result.data["CF9212WLtr2"].sel(species=WLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(target_result.data["CF9212WLtr4"].sel(species=WLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"].sel(species=FRLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_concentrations(target_result.data["CF9212FRLtr4"].sel(species=FRLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_sas(target_result.data["CF9212FRLtr4"].sel(species=FRLspecies), axes[1], cycler=custom_cycler2)
plot_sas(target_result.data["CF9212WLtr4"].sel(species=WLspecies), axes[1], cycler=custom_cycler)
# axes[0].legend()
axes[0].set_xlabel("Time (ps)")
axes[0].set_ylabel("")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("SAS")
axes[1].set_title("SAS")
axes[1].axhline(0, color="k", linewidth=1)
axes[0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
# axes[0].get_legend_handles_labels()


In [ ]:
constFRLRC=1/target_result2.optimized_parameters.get("scale.FRLRCSAS").value
constFRLRC


In [ ]:
import matplotlib.pyplot as plt
from cycler import cycler
from pyglotaran_extras.plotting.plot_concentrations import plot_concentrations
from pyglotaran_extras.plotting.plot_spectra import plot_sas
from pyglotaran_extras.plotting.style import ColorCode
# ColorCode.green,"k",
# myFRLcolors = [ "g","tab:orange",  "r", "tab:grey",ColorCode.cyan,"b", "w", "w"]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.purple, "w", "w","w","w","y",ColorCode.maroon]
# myFRLcolors2 = [ "g","tab:orange",  "r", "k",ColorCode.magenta,ColorCode.indigo, ColorCode.brown,ColorCode.maroon, "w","w","w","y"]
FRLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212FRLRC', 'CF9212Chlf1','CF9212Chlf2']
WLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212WLRC' ]
fig, axes = plt.subplots(2, 2, figsize=(15, 8))
custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
# plot_concentrations(target_result.data["CF9212WLtr2"].sel(species=WLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(target_result.data["CF9212WLtr4"].sel(species=WLspecies), axes[0,0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"].sel(species=FRLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_concentrations(target_result.data["CF9212FRLtr4"].sel(species=FRLspecies), axes[0,0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_sas(target_result.data["CF9212FRLtr4"].sel(species=FRLspecies), axes[0,1], cycler=custom_cycler2)
plot_sas(target_result.data["CF9212WLtr4"].sel(species=WLspecies), axes[0,1], cycler=custom_cycler)
# axes[0].legend()
axes[0,0].set_xlabel("")
axes[0,0].set_ylabel("")
axes[0,0].axhline(0, color="k", linewidth=1)
axes[0,1].set_xlabel("")
axes[0,1].set_ylabel("SAS")
axes[0,1].set_title("SAS")
axes[0,1].axhline(0, color="k", linewidth=1)
axes[0,0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[0,1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
FRLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212FRLRC', 'CF9212Chlf1','CF9212Chlf2', 'CF9212FRLRP1', 'CF9212FRLRP2']
WLspecies=['CF9212Bulk', 'CF9212Red1', 'CF9212Red2', 'CF9212WLRC', 'CF9212WLRP1','CF9212WLRP2', ]
custom_cycler = cycler(color=myFRLcolors, linestyle=["--"] * 7)
# plot_concentrations(target_result.data["CF9212WLtr2"].sel(species=WLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
plot_concentrations(target_result.data["CF9212WLtr4"].sel(species=WLspecies), axes[1,0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler)
custom_cycler2 = cycler(color=myFRLcolors2)
# plot_concentrations(target_result.data["CF9212FRLtr2"].sel(species=FRLspecies), axes[0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
plot_concentrations(target_result.data["CF9212FRLtr4"].sel(species=FRLspecies), axes[1,0], center_λ=0, linlog=True, linthresh=100, cycler=custom_cycler2)
# axes[0].get_legend_handles_labels()
target_result.data["FRLRCSAS"].data.plot(ax=axes[1,1],color="k",linestyle="--")
target_result.data["FRLRCSAS"].fitted_data.plot(ax=axes[1,1],color="k",linestyle="-");
axes[1,1].convert_yunits(constFRLRC)
axes[1,1].set_title("FRL-RC guide")
axes[1,1].set_xlabel("Wavelength (nm)")
axes[1,0].set_xlabel("Time (ps)")
axes[1,0].set_ylabel("")
axes[1,1].set_ylabel("")
axes[1,0].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1,1].annotate("D", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)


In [ ]:
fig, axes = plot_svd_of_residual(target_result2.data["CF9212WLtr1"],target_result2.data["CF9212WLtr2"],target_result2.data["CF9212WLtr4"],linlog=True,linthresh=100,loc1=57,loc2=57,loc3=214)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


In [ ]:
fig, axes = plot_svd_of_residual(target_result1.data["CF9212FRLtr1"],target_result1.data["CF9212FRLtr2"],target_result1.data["CF9212FRLtr4"],linlog=True,linthresh=100,loc1=57,loc2=57,loc3=214)
fig, axes = plot_svd_of_residual(target_result2.data["CF9212FRLtr1"],target_result2.data["CF9212FRLtr2"],target_result2.data["CF9212FRLtr4"],linlog=True,linthresh=100,loc1=57,loc2=57,loc3=214,colors=["k","r","b"])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


## Comparison of the estimated SAS (orange) and the guidance spectra (blue)
The guidance spectra of Red1, Red2, Bulk and WLRC are the shapes estimated in the WL target

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
target_result.data["Red1SAS"].data.plot(ax=axes[0,0])
target_result.data["Red1SAS"].fitted_data.plot(ax=axes[0,0]);
target_result.data["Red2SAS"].data.plot(ax=axes[0,1])
target_result.data["Red2SAS"].fitted_data.plot(ax=axes[0,1]);
target_result.data["FRLRCSAS"].data.plot(ax=axes[1,0])
target_result.data["FRLRCSAS"].fitted_data.plot(ax=axes[1,0]);
target_result.data["BulkSAS"].data.plot(ax=axes[1,1])
target_result.data["BulkSAS"].fitted_data.plot(ax=axes[1,1]);
axes[1,0].set_xlabel("Wavelength (nm)")
axes[1,0].set_ylabel("")
axes[0,0].set_ylabel("")
axes[1,1].set_xlabel("Wavelength (nm)")
axes[1,1].set_ylabel("")
axes[0,1].set_ylabel("")
axes[0,0].set_title("Red Chl a1")
axes[0,1].set_title("Red Chl a2")
axes[1,0].set_title("FRL-RC")
axes[1,1].set_title("Bulk Chl a")
# axes[1].axhline(0, color="k", linewidth=1)
axes[0,0].annotate("A", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[0,1].annotate("B", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1,0].annotate("C", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)
axes[1,1].annotate("D", xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16)


In [ ]:
fig, axes = plt.subplots(1,1, figsize=(5,5))
target_result.data["WLRCSAS"].data.plot(ax=axes)
target_result.data["WLRCSAS"].fitted_data.plot(ax=axes);
axes.set_xlabel("Wavelength (nm)")
axes.set_ylabel("")
axes.set_title("WL-RC")


In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(3, 2))
target_result.data["CF9212FRLtr2"].data.plot(x="time",ax=axis,  cmap='PuRd')
axis.set_xlabel("Time (ps)")
axis.set_ylabel("Wavelength (nm)")


In [ ]:
show_a_matrixes(target_result2)


In [ ]:
target_result2.root_mean_square_error


## Create the target WL&FRL guidance data sets
these can be used for spectral fits and possible further refinement

In [ ]:
from glotaran.io import save_dataset
from glotaran.utils.io import create_clp_guide_dataset

for species in target_result.data["CF9212WLtr2"].species:
    clp_guide = create_clp_guide_dataset(target_result.data["CF9212WLtr2"], species.item())
    string_in_string = "guide/20230913PSI_CF9212_streak_target_dispRP2_{}.ascii".format(species.item())
    save_dataset(clp_guide.data, string_in_string,allow_overwrite=True)
for species in target_result.data["CF9212FRLtr2"].species:
    clp_guide = create_clp_guide_dataset(target_result.data["CF9212FRLtr2"], species.item())
    string_in_string = "guide/20230913PSI_CF9212_streak_target_dispRP2_{}.ascii".format(species.item())
    save_dataset(clp_guide.data, string_in_string,allow_overwrite=True)


## Overview plots per dataset

### WL data

NB plot_overview with linlog=True is very slow, but it can be more informative because it zooms in on the early events

In [ ]:
from cycler import cycler

fig,axes=plot_overview(
    target_result.data["CF9212WLtr1"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    # linlog=True,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
)


In [ ]:
fig,axes=plot_overview(
    target_result.data["CF9212WLtr2"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
)


In [ ]:
fig,axes=plot_overview(
    target_result.data["CF9212WLtr4"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
);


### FRL data

In [ ]:
from cycler import cycler

fig,axes=plot_overview(
    target_result.data["CF9212FRLtr1"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    # linlog=True,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors2),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
)


In [ ]:
fig,axes=plot_overview(
    target_result.data["CF9212FRLtr2"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors2),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
)


In [ ]:
fig,axes=plot_overview(
    target_result.data["CF9212FRLtr4"],
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=100,
    cycler=cycler(color=myFRLcolors2),use_svd_number=True,das_cycler=PlotStyle().cycler,svd_cycler=PlotStyle().cycler,
);


## Fit quality of the target analysis of the WL+FRL data
overlays of traces and fits, first of 16 wavelengths, then of 6 selected wavelengths

In [ ]:
CF9212target_result_streak = (
    target_result.data["CF9212WLtr1"],
    target_result.data["CF9212WLtr2"],
    target_result.data["CF9212WLtr4"],
    target_result.data["CF9212FRLtr1"],
    target_result.data["CF9212FRLtr2"],
    target_result.data["CF9212FRLtr4"],
)


In [ ]:
import numpy as np
# wavelengths = select_plot_wavelengths(
#     CF9212target_result_streak, equidistant_wavelengths=True
# )
wavelengths =np.linspace(665, 795, num=16)
plot_fitted_traces(CF9212target_result_streak, wavelengths, linlog=True, linthresh=100);


In [ ]:
import warnings
from pyglotaran_extras.plotting.style import ColorCode as cc

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fig, ax_ = plot_fitted_traces(
        CF9212target_result_streak,
        [685,705,715,740,750,795],
        linlog=True,
        linthresh=10,
        axes_shape=(3, 2),
        figsize=(8,6),
        title="",
        per_axis_legend=True,
        cycler=cycler(
            color=[
                cc.grey,
                cc.black,
                cc.orange,
                cc.red,
                cc.cyan,
                cc.blue,
                cc.green,
                "g",
                cc.magenta,
                cc.indigo,
                cc.brown,
                cc.maroon,
            ]
        ),
    )
    handles, labels = ax_.flatten()[0].get_legend_handles_labels()
    for i in range(len(handles)):
        if i == 1:
            labels[i] = "WL TR1"
        elif i == 3:
            labels[i] = "WL TR2"
        elif i == 5:
            labels[i] = "WL TR4"
        elif i == 7:
            labels[i] = "FRL TR1"
        elif i == 9:
            labels[i] = "FRL TR2"
        elif i == 11:
            labels[i] = "FRL TR4"
        else:
            labels[i] = "_Hidden"
    for idx, ax in enumerate(ax_.flatten()):
        ax.set_ylabel(ax.title.get_text().replace("spectral = ", ""))
        if idx > 1:
            ax.set_xlabel("Time (ps)")
        else:
            ax.set_xlabel("")
        ax.set_title("")
        if ax.get_legend() is not None:
            ax.get_legend().remove()
        for line in ax.lines:
            # line.set_linewidth(0.5)  # Set the line width here
            line.set_linewidth(1)  # Set the line width here
    fig.legend(
        handles,
        labels,
        bbox_to_anchor=(0.5, -0.05),
        loc="lower center",
        ncol=len(handles),
    )
    fig.tight_layout()
